In [1]:
import os
import torch
import numpy as np
from datasets import ADNI
from torch_geometric.data import DataLoader
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import SplineConv, graclus, voxel_grid, max_pool, max_pool_x

seed = 128
rng = np.random.RandomState(seed)

data = ADNI(root='./data/imaging/')
loader = DataLoader(data, batch_size=5, shuffle=True)
d = data.data

Processing...
Done!


In [2]:
batch_size = 5
num_epochs = 5
filters    = [5, 11]
khops      = [12,14]
fc_size    = 43

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SplineConv(d.num_features, 32, dim=2, kernel_size=5)
        self.conv2 = SplineConv(32, 64, dim=2, kernel_size=5)
        self.fc1 = torch.nn.Linear(86*64, 60)
        self.fc2 = torch.nn.Linear(60, d.num_classes)

    def forward(self, data):
        x = F.elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        #print('conv1:', x.shape)
        x = F.elu(self.conv2(x, data.edge_index, data.edge_attr))
        #print('conv2:', x.shape)
        #print(data.num_graphs, data.num_nodes, data.num_graphs)
        x = x.view([data.num_graphs, (data.num_nodes/data.num_graphs)*64])
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(num_epochs):
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
    if epoch % round(num_epochs/10) == 0:
        print('{} (epoch {}/{})'.format(loss, epoch,num_epochs))
print('Done')

ZeroDivisionError: integer division or modulo by zero

In [ ]:
model.eval()
predictions, target = [], []
for data in loader:
    pred = model(data)
    
    predictions.extend(torch.Tensor.numpy(
                       torch.argmax(pred, dim=1)))
    target.extend(torch.Tensor.numpy(data.y))

In [ ]:
predictions